In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
import pickle
import os
from sklearn.model_selection import train_test_split

In [2]:
df=pd.read_excel("2017 장애인실태조사_최종공개.xlsx")
df.head()

,조사표 종류,가구원 일련번호(장애인),가구원 일련번호(응답자),조사지역(시도),응답자 유형,대리응답이유,장애유형확인1,장애유형확인2,장애유형확인3,개인번호,...,"보육,교육비","보호,간병비",재활기관이용료,통신비,"장애인보조기기 구입,유지비",부모사후 및 노후대비비,기타.4,월평균 총 추가비용,wg_p,ws_p
0,2,1,1,21,NaN,NaN,1,NaN,NaN,1,...,0,0,0,0,0,0,0,0,444.377794,1.132070
1,2,1,1,21,NaN,NaN,1,NaN,NaN,1,...,0,0,0,0,0,0,0,0,444.377794,1.132070
2,2,1,1,21,NaN,NaN,1,NaN,NaN,1,...,0,0,0,0,0,0,0,0,444.377794,1.132070
3,2,2,2,21,NaN,NaN,1,NaN,NaN,2,...,0,0,0,0,0,0,0,28,444.377794,1.132070
4,2,3,3,21,NaN,NaN,8,NaN,NaN,3,...,0,0,25,0,0,0,0,33,575.147902,1.465212


In [878]:
df.columns[209:235]

Index(['만성질환 유무', '01)만성질환명(고혈압)', '02)만성질환명(뇌졸증,중풍)', '03)만성질환명(심근경색증)',
       '04)만성질환명(협심증)', '05)만성질환명(이상지혈증)', '06)만성질환명(당뇨병)', '07)만성질환명(갑상선장애)',
       '08)만성질환명(천식)', '09)만성질환명(폐결핵)', '10)만성질환명(폐질환(만성기관 지염,폐기종)',
       '11)만성질환명(위십이지장궤양)', '12)만성질환명(B형간염)', '13)만성질환명(C형간염)',
       '14)만성질환명(간경변증)', '15)만성질환명(신부전)', '16)만성질환명(골관절염(퇴행성\n관절염))',
       '17)만성질환명(류마티스 관절염)', '18)만성질환명(골다공증)', '19)만성질환명(척추측만증)',
       '20)만성질환명(허리목통증)', '21)만성질환명(피부염)', '22)만성질환명(백내장)', '23)만성질환명(우울증)',
       '24)만성질환명(암)', '25)만성질환명(기타)'],
      dtype='object')

In [183]:
df1=df[['장애유형확인1', 
   '성별', '생년','장애등록 연도','장애등급',
   #'물리치료-이용여부','작업치료-이용여부','언어치료-이용여부','음악치료-이용여부','놀이치료-이용여부','미술치료-이용여부','심리행동치료-이용여부','기타-이용여부',
   '장애부위(상지)', '장애부위(하지)', '장애부위(척추)', '가장 불편한부위',
   '장애발생시 연령','장애주된 원인', '질병명', '주된 진단명','03)만성질환명(심근경색증)']]
df1 #6549건 

,장애유형확인1,성별,생년,장애등록 연도,장애등급,장애부위(상지),장애부위(하지),장애부위(척추),가장 불편한부위,장애발생시 연령,장애주된 원인,질병명,주된 진단명,03)만성질환명(심근경색증)
0,1,1,1970,2008,6,0.0,1.0,0.0,36.0,38.0,64.0,0.0,12.0,2
1,1,1,1949,2010,6,1.0,0.0,0.0,14.0,20.0,69.0,0.0,8.0,2
2,1,1,1976,2015,6,0.0,1.0,0.0,37.0,23.0,69.0,0.0,12.0,0
3,1,2,1958,2011,5,0.0,0.0,1.0,38.0,51.0,48.0,175.0,11.0,2
4,8,2,1984,2016,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6544,1,2,1960,2015,6,0.0,0.0,1.0,31.0,55.0,48.0,175.0,11.0,2
6545,1,1,1944,2009,5,0.0,1.0,0.0,36.0,63.0,48.0,171.0,6.0,2
6546,1,1,1952,2004,6,0.0,1.0,0.0,23.0,2.0,50.0,190.0,1.0,0
6547,9,1,1958,2000,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2


In [184]:
df2=df1[df1['03)만성질환명(심근경색증)']!=0].reset_index(drop=True)
df2

,장애유형확인1,성별,생년,장애등록 연도,장애등급,장애부위(상지),장애부위(하지),장애부위(척추),가장 불편한부위,장애발생시 연령,장애주된 원인,질병명,주된 진단명,03)만성질환명(심근경색증)
0,1,1,1970,2008,6,0.0,1.0,0.0,36.0,38.0,64.0,0.0,12.0,2
1,1,1,1949,2010,6,1.0,0.0,0.0,14.0,20.0,69.0,0.0,8.0,2
2,1,2,1958,2011,5,0.0,0.0,1.0,38.0,51.0,48.0,175.0,11.0,2
3,8,2,1984,2016,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
4,9,1,1972,2015,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5258,1,1,1960,2010,6,2.0,2.0,1.0,38.0,48.0,48.0,175.0,11.0,2
5259,1,2,1960,2015,6,0.0,0.0,1.0,31.0,55.0,48.0,175.0,11.0,2
5260,1,1,1944,2009,5,0.0,1.0,0.0,36.0,63.0,48.0,171.0,6.0,2
5261,9,1,1958,2000,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2


In [185]:
conditionlist = [
    (df2['03)만성질환명(심근경색증)'] ==1) ,
    (df2['03)만성질환명(심근경색증)'] == 2) ]
choicelist = [0,1]
df2['03)만성질환명(심근경색증)'] = np.select(conditionlist, choicelist)

In [186]:
df3=df2.dropna().reset_index(drop=True) #결측치 삭제 
df3#3296건 

,장애유형확인1,성별,생년,장애등록 연도,장애등급,장애부위(상지),장애부위(하지),장애부위(척추),가장 불편한부위,장애발생시 연령,장애주된 원인,질병명,주된 진단명,03)만성질환명(심근경색증)
0,1,1,1970,2008,6,0.0,1.0,0.0,36.0,38.0,64.0,0.0,12.0,1
1,1,1,1949,2010,6,1.0,0.0,0.0,14.0,20.0,69.0,0.0,8.0,1
2,1,2,1958,2011,5,0.0,0.0,1.0,38.0,51.0,48.0,175.0,11.0,1
3,1,2,1955,2009,5,0.0,0.0,1.0,38.0,53.0,48.0,175.0,11.0,1
4,1,2,1936,2010,3,0.0,3.0,0.0,36.0,62.0,48.0,171.0,6.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2719,1,1,1935,1997,3,0.0,1.0,0.0,36.0,18.0,68.0,0.0,12.0,1
2720,1,1,1952,2015,5,0.0,0.0,1.0,38.0,61.0,69.0,0.0,12.0,1
2721,1,1,1960,2010,6,2.0,2.0,1.0,38.0,48.0,48.0,175.0,11.0,1
2722,1,2,1960,2015,6,0.0,0.0,1.0,31.0,55.0,48.0,175.0,11.0,1


In [205]:
df3[df3['03)만성질환명(심근경색증)'] ==0].index

Int64Index([  42,   46,   53,   76,  112,  135,  174,  186,  218,  230,
            ...
            2539, 2554, 2575, 2587, 2594, 2621, 2633, 2674, 2692, 2712],
           dtype='int64', length=108)

### 결측치가 없는 관측치 하나 미리 빼놓기 (테스트용)

In [243]:
out=df3.loc[[46]]
out

,장애유형확인1,성별,생년,장애등록 연도,장애등급,장애부위(상지),장애부위(하지),장애부위(척추),가장 불편한부위,장애발생시 연령,장애주된 원인,질병명,주된 진단명,03)만성질환명(심근경색증)
46,1,1,1947,1999,3,0.0,0.0,1.0,38.0,36.0,69.0,0.0,12.0,0


In [244]:
df4=df3.drop(out.index).reset_index(drop=True)

In [245]:
x_data=df4[['장애유형확인1', 
   '성별', '생년','장애등록 연도','장애등급',
   #'물리치료-이용여부','작업치료-이용여부','언어치료-이용여부','음악치료-이용여부','놀이치료-이용여부','미술치료-이용여부','심리행동치료-이용여부','기타-이용여부',
   '장애부위(상지)', '장애부위(하지)', '장애부위(척추)', '가장 불편한부위',
   '장애발생시 연령','장애주된 원인', '질병명', '주된 진단명']]

In [246]:
y_data=df4[['03)만성질환명(심근경색증)']]

In [247]:
y_data.value_counts()

03)만성질환명(심근경색증)
1                  2616
0                   107
dtype: int64

In [248]:
#!pip install SMOTE

In [249]:
#!pip install -U imbalanced-learn

In [250]:
y_sm.value_counts()

03)만성질환명(심근경색증)
0                  1837
1                  1837
dtype: int64

In [251]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=777)

### 오버샘플링

In [252]:
from imblearn.over_sampling import SMOTE

smote = SMOTE()
x_sm, y_sm = smote.fit_resample(x_train, y_train)

In [253]:
transformer = make_column_transformer(
        (OneHotEncoder(handle_unknown = "ignore"), ['장애유형확인1', 
   '성별','장애등급',
 #  '물리치료-이용여부','작업치료-이용여부','언어치료-이용여부','음악치료-이용여부','놀이치료-이용여부','미술치료-이용여부','심리행동치료-이용여부','기타-이용여부',
   '장애부위(상지)', '장애부위(하지)', '장애부위(척추)', '가장 불편한부위', '장애주된 원인', '질병명', '주된 진단명']),
        remainder='passthrough')
transformer = make_pipeline(transformer, MaxAbsScaler())
transformer.fit(x_sm)
x_sm = transformer.transform(x_sm)
x_test = transformer.transform(x_test)

In [254]:
if not os.path.exists('models/disabled_chronic_disease/'):
    os.makedirs('models/disabled_chronic_disease/')

with open('models/disabled_chronic_disease/transformer.pkl', 'wb') as f:
    pickle.dump(transformer, f)

In [255]:
#모델 생성

model = SVC(kernel='rbf', C=5.0, gamma=0.1, probability=True)#,class_weight=class_weight)

In [256]:
#모델 학습
model.fit(x_sm, y_sm)

#모델 검증
print('training data fit :', model.score(x_sm, y_sm)) 

print('test data fit :', model.score(x_test, y_test)) 

with open('models/disabled_chronic_disease/transformer.pkl', 'wb') as f:
    pickle.dump(model, f)

C:\python\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


training data fit : 0.9243331518780621
test data fit : 0.8996328029375765


In [262]:
x_test=out.drop(['03)만성질환명(심근경색증)'],axis=1)
x_test

,장애유형확인1,성별,생년,장애등록 연도,장애등급,장애부위(상지),장애부위(하지),장애부위(척추),가장 불편한부위,장애발생시 연령,장애주된 원인,질병명,주된 진단명
46,1,1,1947,1999,3,0.0,0.0,1.0,38.0,36.0,69.0,0.0,12.0


In [263]:
labels = ['유', '무']

In [264]:
x_test = transformer.transform(x_test)

y_predict = model.predict(x_test)
label = labels[y_predict[0]]
y_predict = model.predict_proba(x_test)
confidence = y_predict[0][y_predict[0].argmax()]

print(label, confidence) 

무 0.9475934874682628


---

In [240]:
x_test=pd.DataFrame({'장애유형확인1': 1, # 지체장애
             '성별': 2, # 여
              '생년': 1988,
              '장애등록 연도': 2014,
              '장애등급': 2, # 2등급
              '물리치료-이용여부': 1, # 예
              '작업치료-이용여부': 2, # 아니오
              '언어치료-이용여부': 2, # 아니오
              '음악치료-이용여부': 2, # 아니오
              '놀이치료-이용여부': 1, # 예
              '미술치료-이용여부': 2, # 아니오
              '심리행동치료-이용여부': 2, # 아니오
              '기타-이용여부': 2, # 아니오
             '장애부위(상지)': 1, # 우측
             '장애부위(하지)': 2, # 좌측
             '장애부위(척추)': 1, 
             '가장 불편한부위': 38, # 척추(디스크)
              '장애발생시 연령': 27,
             '장애주된 원인': 10, # 유전 및 염색체 이상
             '질병명': 178, # 골다공증 
             '주된 진단명': 5}, # 척수염
index = [0])

In [260]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_validate

rf=RandomForestClassifier(criterion='entropy')
cv_rf = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

scores = cross_validate(rf, x_sm, y_sm, scoring='f1', cv=cv_rf, n_jobs=-1)

print('3-Fold CV f1 score: %.4f' % np.mean(scores['test_score']))

3-Fold CV f1 score: 0.9664
